# About
This notebook contains a first simple ML model

In [1]:
# Necessary to import custom modules
import os
os.chdir("/home/jovyan/work")

from neomodel import db
import pandas as pd

from src.utils import init_connection

In [2]:
init_connection()

In [11]:
cypher = """
MATCH 
    (t:Team)-[:SCORED]->(s:Score)-[:IN_GAME]->(g:Game)<-[:IN_GAME]-(s2:Score)<-[:SCORED]-(t2:Team), (sea:Season), (arena_game:Arena), (arena_t:Arena)
WHERE 
    (g)-[:TOOK_PLACE_IN]->(sea)
AND
	(g)-[:LOCATED_IN]->(arena_game)
AND
	(t)-[:HAVE_HOME_COURT_AT]->(arena_t)
RETURN 
    t.name as team, 
    sum(
    	CASE WHEN s.score > s2.score 
        AND g.game_type = "regular_season" 
        AND arena_game.name = arena_t.name
        THEN 1 ELSE 0 END) 
    as wins_reg_home,
    sum(
    	CASE WHEN s.score > s2.score 
        AND g.game_type = "regular_season" 
        AND arena_game.name <> arena_t.name
        THEN 1 ELSE 0 END) 
    as wins_reg_away,
    sum(
    	CASE WHEN s.score < s2.score 
        AND g.game_type = "regular_season" 
        AND arena_game.name = arena_t.name
        THEN 1 ELSE 0 END) 
    as losses_reg_home,
    sum(
    	CASE WHEN s.score < s2.score
        AND g.game_type = "regular_season" 
        AND arena_game.name <> arena_t.name
        THEN 1 ELSE 0 END) as losses_reg_away,
	sea.name as season
ORDER BY 
    team, season
"""

In [12]:
data, columns = db.cypher_query(cypher)

In [13]:
df_results = pd.DataFrame(data=data, columns=columns)

In [14]:
df_test = df_results.pivot(index="team", columns="season")

In [15]:
cols = []
for a, b in zip(df_test.columns.get_level_values(0), df_test.columns.get_level_values(1)):
    cols.append(a+"_"+b)

In [16]:
cols

['wins_reg_home_2015/2016',
 'wins_reg_home_2016/2017',
 'wins_reg_home_2017/2018',
 'wins_reg_away_2015/2016',
 'wins_reg_away_2016/2017',
 'wins_reg_away_2017/2018',
 'losses_reg_home_2015/2016',
 'losses_reg_home_2016/2017',
 'losses_reg_home_2017/2018',
 'losses_reg_away_2015/2016',
 'losses_reg_away_2016/2017',
 'losses_reg_away_2017/2018']

In [17]:
df_test.columns = cols

In [18]:
df_test.head()

,wins_reg_home_2015/2016,wins_reg_home_2016/2017,wins_reg_home_2017/2018,wins_reg_away_2015/2016,wins_reg_away_2016/2017,wins_reg_away_2017/2018,losses_reg_home_2015/2016,losses_reg_home_2016/2017,losses_reg_home_2017/2018,losses_reg_away_2015/2016,losses_reg_away_2016/2017,losses_reg_away_2017/2018
team,,,,,,,,,,,,
Atlanta Hawks,27,23,16,21,20,8,14,18,25,20,21,33
Boston Celtics,28,30,27,20,23,28,13,11,14,21,18,13
Brooklyn Nets,14,13,15,7,7,13,27,28,26,34,34,28
Charlotte Hornets,30,22,21,18,14,15,11,19,20,23,27,26
Chicago Bulls,26,25,17,16,16,10,15,16,24,25,25,31


#### Extract Matchup

```
MATCH 
	(t:Team)-[:SCORED]->(s:Score)-[:IN_GAME]->(g:Game)<-[:IN_GAME]-(s2:Score)<-[:SCORED]-(t2:Team), (sea:Season)
WHERE 
    (g)-[:TOOK_PLACE_IN]->(sea)
AND
	sea.name = "2017/2018"
AND
	g.game_type = "regular_season"
WITH g, collect(t.name) as teams, collect(s.score) as scores
RETURN
	HEAD(teams) as teamA, HEAD(TAIL(scores)) as scoreA, HEAD(TAIL(teams)) as teamB, HEAD(scores) as scoreB
```

In [11]:
cypher = """
MATCH 
	(t:Team)-[:SCORED]->(s:Score)-[:IN_GAME]->(g:Game)<-[:IN_GAME]-(s2:Score)<-[:SCORED]-(t2:Team), (sea:Season)
WHERE 
    (g)-[:TOOK_PLACE_IN]->(sea)
AND
	sea.name = "2017/2018"
AND
	g.game_type = "regular_season"
WITH g, collect(t.name) as teams, collect(s.score) as scores
RETURN
	HEAD(teams) as teamA, HEAD(TAIL(scores)) as scoreA, HEAD(TAIL(teams)) as teamB, HEAD(scores) as scoreB
"""

In [12]:
data, cols = db.cypher_query(cypher)

In [13]:
df_matches = pd.DataFrame(data=data, columns=cols)

In [14]:
df_matches["teamAWinner"] =df_matches["scoreA"] > df_matches["scoreB"]

In [15]:
df_test.head().reset_index()

,team,wins_reg_home_2015/2016,wins_reg_home_2016/2017,wins_reg_home_2017/2018,wins_reg_away_2015/2016,wins_reg_away_2016/2017,wins_reg_away_2017/2018,losses_reg_home_2015/2016,losses_reg_home_2016/2017,losses_reg_home_2017/2018,losses_reg_away_2015/2016,losses_reg_away_2016/2017,losses_reg_away_2017/2018
0,Atlanta Hawks,27,23,16,21,20,8,14,18,25,20,21,33
1,Boston Celtics,28,30,27,20,23,28,13,11,14,21,18,13
2,Brooklyn Nets,14,13,15,7,7,13,27,28,26,34,34,28
3,Charlotte Hornets,30,22,21,18,14,15,11,19,20,23,27,26
4,Chicago Bulls,26,25,17,16,16,10,15,16,24,25,25,31


In [16]:
df_matches.head()

,teamA,scoreA,teamB,scoreB,teamAWinner
0,Boston Celtics,80,Philadelphia 76ers,89,False
1,Charlotte Hornets,111,Toronto Raptors,129,False
2,Toronto Raptors,96,Detroit Pistons,91,True
3,Milwaukee Bucks,110,Toronto Raptors,129,False
4,Oklahoma City Thunder,105,Miami Heat,99,True


In [80]:
pd.merge(df_matches, df_test.reset_index(), left_on="teamA", right_on="team").head()

,teamA,scoreA,teamB,scoreB,scoreMargin,team,wins_reg_home_2015/2016,wins_reg_home_2016/2017,wins_reg_home_2017/2018,wins_reg_away_2015/2016,wins_reg_away_2016/2017,wins_reg_away_2017/2018,losses_reg_home_2015/2016,losses_reg_home_2016/2017,losses_reg_home_2017/2018,losses_reg_away_2015/2016,losses_reg_away_2016/2017,losses_reg_away_2017/2018
0,Boston Celtics,80,Philadelphia 76ers,89,9,Boston Celtics,28,30,27,20,23,28,13,11,14,21,18,13
1,Boston Celtics,102,Cleveland Cavaliers,88,14,Boston Celtics,28,30,27,20,23,28,13,11,14,21,18,13
2,Boston Celtics,97,Indiana Pacers,99,2,Boston Celtics,28,30,27,20,23,28,13,11,14,21,18,13
3,Boston Celtics,134,Charlotte Hornets,106,28,Boston Celtics,28,30,27,20,23,28,13,11,14,21,18,13
4,Boston Celtics,89,Miami Heat,90,1,Boston Celtics,28,30,27,20,23,28,13,11,14,21,18,13


In [19]:
df_ml = pd.merge(
    pd.merge(df_matches, df_test.reset_index(), left_on="teamA", right_on="team"),
    df_test.reset_index(), 
    left_on="teamB", 
    right_on="team",
    suffixes=["_A", "_B"]
).drop(
    ["scoreA", "scoreB", "teamA", "teamB", "team_A", "team_B", "wins_reg_home_2017/2018_A", "wins_reg_away_2017/2018_A", "wins_reg_home_2017/2018_B", "wins_reg_away_2017/2018_B", "losses_reg_home_2017/2018_A", "losses_reg_away_2017/2018_A", "losses_reg_home_2017/2018_B", "losses_reg_away_2017/2018_B"], 
    axis=1)

In [20]:
df_ml.head()

,teamAWinner,wins_reg_home_2015/2016_A,wins_reg_home_2016/2017_A,wins_reg_away_2015/2016_A,wins_reg_away_2016/2017_A,losses_reg_home_2015/2016_A,losses_reg_home_2016/2017_A,losses_reg_away_2015/2016_A,losses_reg_away_2016/2017_A,wins_reg_home_2015/2016_B,wins_reg_home_2016/2017_B,wins_reg_away_2015/2016_B,wins_reg_away_2016/2017_B,losses_reg_home_2015/2016_B,losses_reg_home_2016/2017_B,losses_reg_away_2015/2016_B,losses_reg_away_2016/2017_B
0,False,28,30,20,23,13,11,21,18,7,17,3,11,34,24,38,30
1,True,28,30,20,23,13,11,21,18,7,17,3,11,34,24,38,30
2,False,30,22,18,14,11,19,23,27,7,17,3,11,34,24,38,30
3,False,30,22,18,14,11,19,23,27,7,17,3,11,34,24,38,30
4,True,32,28,24,23,9,13,17,18,7,17,3,11,34,24,38,30


In [21]:
from sklearn.linear_model import LogisticRegression

In [22]:
lr = LogisticRegression()

In [23]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_test, y_train, y_test = train_test_split(
    df_ml.drop("teamAWinner", axis=1), 
    df_ml["teamAWinner"]
)

lr.fit(X_train, y_train)

lr.score(X_test, y_test)

0.64610389610389607

# Average Win Margin per Team 

## Validierung der eigentlichen Cypher-Query

In [95]:
cypher = """
MATCH 
	(t:Team)-[:SCORED]->(s:Score)-[:IN_GAME]->(g:Game)<-[:IN_GAME]-(s2:Score)<-[:SCORED]-(t2:Team), (sea:Season)
WHERE 
    (g)-[:TOOK_PLACE_IN]->(sea)
AND
	sea.name = "2017/2018"
AND
	g.game_type = "regular_season"
WITH g, collect(t.name) as teams, collect(s.score) as scores
RETURN
	HEAD(teams) as teamA, HEAD(TAIL(scores)) as scoreA, HEAD(TAIL(teams)) as teamB, HEAD(scores) as scoreB
"""

In [96]:
data, cols = db.cypher_query(cypher)

In [97]:
df_matches = pd.DataFrame(data=data, columns=cols)

In [98]:
df_matches.head()

,teamA,scoreA,teamB,scoreB
0,Boston Celtics,80,Philadelphia 76ers,89
1,Charlotte Hornets,111,Toronto Raptors,129
2,Toronto Raptors,96,Detroit Pistons,91
3,Milwaukee Bucks,110,Toronto Raptors,129
4,Oklahoma City Thunder,105,Miami Heat,99


In [99]:
df_matches2 = df_matches

In [100]:
df_matches2["scoreMargin"] = abs(df_matches2["scoreA"]-df_matches2["scoreB"])

In [101]:
df_matches2.head()

,teamA,scoreA,teamB,scoreB,scoreMargin
0,Boston Celtics,80,Philadelphia 76ers,89,9
1,Charlotte Hornets,111,Toronto Raptors,129,18
2,Toronto Raptors,96,Detroit Pistons,91,5
3,Milwaukee Bucks,110,Toronto Raptors,129,19
4,Oklahoma City Thunder,105,Miami Heat,99,6


In [107]:
a = ["Houston Rockets"]

In [108]:
df_onlyBOSmatches = df_matches2[(df_matches2['teamA'].isin(a)) | (df_matches2['teamB'].isin(a)) ]

In [109]:
df_onlyBOSmatches["scoreMargin"].mean()

12.25609756097561

## Actual Cypher Query

In [34]:
cypher = """
MATCH 
    (t:Team)-[:SCORED]->(s:Score)-[:IN_GAME]->(g:Game)<-[:IN_GAME]-(s2:Score)<-[:SCORED]-(t2:Team), (sea:Season)
WHERE 
    (g)-[:TOOK_PLACE_IN]->(sea)
AND
	g.game_type= "regular_season"
RETURN 
    t.name as team, 
    avg(abs(s.score - s2.score)) as scoreMargin,
    sea.name as season
ORDER BY 
    team, season

"""

In [35]:
data, cols = db.cypher_query(cypher)

In [36]:
df_scoreMargin = pd.DataFrame(data=data, columns=cols)

In [37]:
df_scoreMargin.head()

,team,scoreMargin,season
0,Atlanta Hawks,11.634146,2015/2016
1,Atlanta Hawks,11.170732,2016/2017
2,Atlanta Hawks,10.865854,2017/2018
3,Boston Celtics,10.695122,2015/2016
4,Boston Celtics,9.195122,2016/2017


In [38]:
df_scoreMargin_test = df_scoreMargin.pivot(index="team", columns="season")

In [39]:
df_scoreMargin_test.head()

scoreMargin                      
season              2015/2016  2016/2017  2017/2018
team                                               
Atlanta Hawks       11.634146  11.170732  10.865854
Boston Celtics      10.695122   9.195122   9.219512
Brooklyn Nets       10.743902  12.195122  10.231707
Charlotte Hornets   11.134146  10.195122  11.768293
Chicago Bulls        9.865854  12.036585  12.231707

In [40]:
cols = []
for a, b in zip(df_scoreMargin_test.columns.get_level_values(0), df_scoreMargin_test.columns.get_level_values(1)):
    cols.append(a+"_"+b)

In [41]:
cols

['scoreMargin_2015/2016', 'scoreMargin_2016/2017', 'scoreMargin_2017/2018']

In [42]:
df_scoreMargin_test.columns = cols

In [43]:
df_scoreMargin_test.head()

,scoreMargin_2015/2016,scoreMargin_2016/2017,scoreMargin_2017/2018
team,,,
Atlanta Hawks,11.634146,11.170732,10.865854
Boston Celtics,10.695122,9.195122,9.219512
Brooklyn Nets,10.743902,12.195122,10.231707
Charlotte Hornets,11.134146,10.195122,11.768293
Chicago Bulls,9.865854,12.036585,12.231707


In [44]:
df_ml_1 = pd.merge(
    pd.merge(df_matches, df_test.reset_index(), left_on="teamA", right_on="team"),
    df_test.reset_index(), 
    left_on="teamB", 
    right_on="team",
    suffixes=["_A", "_B"]
)

In [45]:
df_ml_2 = pd.merge(
    pd.merge(df_ml_1, df_scoreMargin_test.reset_index(), left_on="teamA", right_on="team"),
    df_scoreMargin_test.reset_index(), 
    left_on="teamB", 
    right_on="team",
    suffixes=["_A", "_B"]
).drop(
    ["scoreA", "scoreB", "teamA", "teamB", "team_A", "team_B", "wins_reg_2017/2018_A", "losses_reg_2017/2018_A", "wins_reg_2017/2018_B", "losses_reg_2017/2018_B", "scoreMargin_2017/2018_A", "scoreMargin_2017/2018_B" ], 
    axis=1)

In [46]:
df_ml_2.head()

,teamAWinner,wins_reg_home_2015/2016_A,wins_reg_home_2016/2017_A,wins_reg_home_2017/2018_A,wins_reg_away_2015/2016_A,wins_reg_away_2016/2017_A,wins_reg_away_2017/2018_A,losses_reg_home_2015/2016_A,losses_reg_home_2016/2017_A,losses_reg_home_2017/2018_A,...,losses_reg_home_2015/2016_B,losses_reg_home_2016/2017_B,losses_reg_home_2017/2018_B,losses_reg_away_2015/2016_B,losses_reg_away_2016/2017_B,losses_reg_away_2017/2018_B,scoreMargin_2015/2016_A,scoreMargin_2016/2017_A,scoreMargin_2015/2016_B,scoreMargin_2016/2017_B
0,False,28,30,27,20,23,28,13,11,14,...,34,24,11,38,30,19,10.695122,9.195122,12.743902,10.865854
1,True,28,30,27,20,23,28,13,11,14,...,34,24,11,38,30,19,10.695122,9.195122,12.743902,10.865854
2,False,30,22,21,18,14,15,11,19,20,...,34,24,11,38,30,19,11.134146,10.195122,12.743902,10.865854
3,False,30,22,21,18,14,15,11,19,20,...,34,24,11,38,30,19,11.134146,10.195122,12.743902,10.865854
4,True,32,28,34,24,23,25,9,13,7,...,34,24,11,38,30,19,9.914634,10.548780,12.743902,10.865854


## Score Margin splitted into home and away

In [126]:
cypher = """
MATCH 
    (t:Team)-[:SCORED]->(s:Score)-[:IN_GAME]->(g:Game)<-[:IN_GAME]-(s2:Score)<-[:SCORED]-(t2:Team), (sea:Season)
WHERE 
    (g)-[:TOOK_PLACE_IN]->(sea)
AND
	g.game_type= "regular_season"
RETURN 
    t.name as team, 
    sea.name as season,
    CASE WHEN s.score < s2.score THEN avg(abs(s.score - s2.score)) ELSE 0 END as scoreMarginWins,
    CASE WHEN s.score > s2.score THEN avg(abs(s.score - s2.score)) ELSE 0 END as scoreMarginLosses

ORDER BY 
    team, season

"""

In [127]:
data, cols = db.cypher_query(cypher)

In [128]:
df_scoreMargin = pd.DataFrame(data=data, columns=cols)

In [129]:
df_scoreMargin.head()

,team,season,scoreMarginWins,scoreMarginLosses
0,Atlanta Hawks,2015/2016,0.000000,9.676471
1,Atlanta Hawks,2015/2016,13.020833,0.000000
2,Atlanta Hawks,2016/2017,0.000000,12.641026
3,Atlanta Hawks,2016/2017,9.837209,0.000000
4,Atlanta Hawks,2017/2018,0.000000,11.534483


In [137]:
df_scoreMargin_home_away = df_scoreMargin.groupby(["team", "season"]).sum().reset_index()

In [138]:
df_scoreMargin_test = df_scoreMargin_home_away.pivot(index="team", columns="season")

In [139]:
cols = []
for a, b in zip(df_scoreMargin_test.columns.get_level_values(0), df_scoreMargin_test.columns.get_level_values(1)):
    cols.append(a+"_"+b)

In [133]:
df_scoreMargin_test.columns = cols

## Validation of numbers

In [134]:
df_scoreMargin_home_away[df_scoreMargin_home_away["team"] == "Houston Rockets"]

,team,season,scoreMarginWins,scoreMarginLosses
30,Houston Rockets,2015/2016,10.268293,9.878049
31,Houston Rockets,2016/2017,13.490909,9.962963
32,Houston Rockets,2017/2018,13.076923,9.117647


(4+5+1+18+7+11+2+3+17+13)

(6+10+6+10+16+8+8)

In [135]:
test = [14,5,1,18,7,11,2,3,17,13,6,10,6,10,16,8,8]

In [136]:
import numpy as np
np.mean(test)

9.117647058823529

# Models

In [140]:
from sklearn.linear_model import LogisticRegression

In [141]:
lr = LogisticRegression()

In [142]:
from sklearn.model_selection import train_test_split

In [148]:
X_train, X_test, y_train, y_test = train_test_split(
    df_ml_2.drop("teamAWinner", axis=1), 
    df_ml_2["teamAWinner"]
)

lr.fit(X_train, y_train)

lr.score(X_test, y_test)

0.72727272727272729

In [144]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

In [145]:
X_train, X_test, y_train, y_test = train_test_split(
    df_ml_2.drop("teamAWinner", axis=1), 
    df_ml_2["teamAWinner"]
)
clf.fit(X_train, y_train) 
clf.score(X_test, y_test)

0.58766233766233766